# MRI analysis pipeline

<img src="pipeline.png" width="900" />

# MRI pre-processing
## fMRI pre-processing pipeline

<img src="fMRI_pipeline.png" width="900" />

## dMRI pre-processing pipeline

<img src="dMRI_pipeline.png" width="700" />

## Co-ordinate systems
### 3D Cartesian

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/Coord_system_CA_0.svg" width="400" />

### 3D Spherical

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Spherical.svg" width="400" />

## Inter-subject co-registration

We would like to put two 3D images from two different subjects on top of each other as well as possible.

In [6]:
# Two images before co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="before_coreg.mov">
</video>
""")

In [7]:
# Two images after co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="after_coreg.mov">
</video>
""")

A standard way to do this is to use one of the brain images as a registration target and move the other into its reference frame. This is typically accomplished in two phases: 1) linear, and 2) non-linear co-registration.

### Linear transformations

All of the linear transformations are global, in a way that they affect each voxel the same way, and can be descrbed using a 4x4 transformation matrix,

$$
T =
\begin{pmatrix}
   0.1 & 0   & 0   & -5   \\
   0   & 0.1 & 0   & -6.7 \\
   0   & 0   & 0.1 & -10  \\
   0   & 0   & 0   & 1
\end{pmatrix}
$$

This example matrix describes a space with isotropic voxel size (of 0.1 units) and a translation of -5 -6.7 -10 units (usually in x,y,z coordinates). The linear trasformations can be split into translation, rotation, scaling, shear, and perspective transformations.


Original image

<img src="orig_img.png" width="500" />

Translation

<table><tr>
<td> <img src="trans_img.png" width="500" /> </td>
<td> 
    $$
T_{trans} =
\begin{pmatrix}
   1 & 0 & 30 \\
   0 & 1 & 30 \\
   0 & 0 & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>

Scaling

<table><tr>
<td> <img src="scaled_img.png" width="500" /> </td>
<td> 
$$
T_{scaling} =
\begin{pmatrix}
   1.3 & 0   & 10 \\
   0   & 0.8 & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>

Rotation

<table><tr>
<td> <img src="rot_img.png" width="500" /> </td>
<td> 
$$
T_{rot} =
\begin{pmatrix}
   \cos(\theta) & -\sin(\theta) & 30 \\
   \sin(\theta) &  \cos(\theta) & 30 \\
   0            & 0             & 1  
\end{pmatrix}
=
\begin{pmatrix}
   0.980 & -0.199 & 40 \\
   0.199 &  0.980 & 20 \\
   0     &  0     & 1  
\end{pmatrix}
,
$$
$$ \theta = .2 $$
</td>
</tr>
</table>


Shear

<table><tr>
<td> <img src="sheared_img.png" width="500" /> </td>
<td> 
$$
T_{shear} =
\begin{pmatrix}
   1   & 0.1 & 10 \\
   0.1 & 1   & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>


Perspective

<table><tr>
<td> <img src="perspective_img.png" width="500" /> </td>
<td> 
$$
T_{perspective} =
\begin{pmatrix}
   1     & 0 & 10 \\
   0     & 1 & 30 \\
   0.001 & 0 & 1  
\end{pmatrix}
$$
</td>
</tr>
</table>


The perspective transformation does not happen in MRI data, so the last row of T is zeros apart from the diagonal 1.

#### How to use the matrices

Each voxel location in the 3D image is represented with a vector in a cartesian coordinate system. A mesh. The transformed vector is a result of matrix-vector multiplication

\begin{gather}
  v_{old} =
  \begin{pmatrix}
      x\\
      y\\
      z
  \end{pmatrix}
  \\
  v_{new} = T v_{old}
\end{gather}

Several transformations can be applied to the same vector, which may be useful in longitudinal studies, particularly the ones with progressive deformations.

![longtudinal](longitudinal_coregistration.png)

$$ v_{new} = T_1 T_2 T_3 T_4 v_{old}$$


#### Additional terminology

Linear co-registration is often divided into two separate parts:
    1. rigid co-registration, where the object experiences translations and rotations but remains intact otherwise; requiring 6 parameters, 3 translation, 3 rotation
    2. affine co-registration, where - 9 or 12 parameters (figure)


### Non-linear co-registration

* talk about the liberties added compared to the linear case

Non-linear registration performs local transformations. It is used primarily to correct for local deformations. As unconstrained non-linear transform can fit anything to everything, we need constraints. One could approximate that deformations are linear in the smaller scale, and use piecewise linear co-registration to correct for deformations. Or, that objects close to each other in the original image should be relatively close in the transformed image, too. The latest is implemented in ANTs software as diffeomorphic mapping.

```shell
antsRegistration --dimensionality 3 --float 0 --output [trnsfrm_nl_,deformed.nii.gz,inverse.nii.gz] --interpolation Linear --winsorize-image-intensities [0.005,0.995] --use-histogram-matching 0 --initial-moving-transform trnsfrm_rigid_0GenericAffine.mat --transform SyN[0.2,2,0] --metric CC[$(SIGMA_FILE),orig.nii.gz,1,4] --convergence [400x300x200,1e-5,10] --shrink-factors 4x2x1 --smoothing-sigmas 2x1x0vox --verbose
```

The product is often a deformation vector field, which describes local deformations of the original image; where to move the voxel center in order to produce the deformed image.

## Metrics


Correlation: 
$$
r =\frac{n \sum{xy} - (\sum{x})(\sum{y})}{\sqrt{\left(n \sum{x^2}-(\sum{x})^2\right)\left(n \sum{y^2}-(\sum{y})^2\right)}}
$$

How to describe a relationship between two variables: Linear relationship: fit a line so that the fit minimizes the sum of distances between the points and the line 


<img src="PNS.png" width="350" />(PNS.png)

    * cross correlation: move one image and find the minimal correlation
    * show two brain images partially on top of each other and and nicely registered + scatter plots from both

* cross correlation - same to same - show images and scatter plots
    * show two brain images partially on top of each other and nicely registered



In [8]:
# Two images before co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="before_coreg.mov">
</video>
""")

<img src="orig_scatter.png" width="300" />

In [13]:
# Two images after co-registration
from IPython.display import HTML
HTML("""
<video width="560" height="560" controls loop>
  <source src="after_coreg.mov">
</video>
""")

<img src="coreg_scatter.png" width="300" />
<img src="both_scatter.png" width="300" />

    * talk about correlating and non-correlating data using the scatters
    * maybe brain + ct?
    * and then the solution


* mutual information - different to different
    * show images, talk about common information
    * show scatterplot -> no correlation, show joint information scatter -> correlation

$$
\mathrm {I}(X;Y) = \sum_{y \in Y}{\sum_{x \in X}{p_{(X,Y)}(x,y) \log{\left(\frac{p_{(X,Y)}(x,y)}{p_X(x) p_Y(y)}\right)}}}
$$

$$
p_{X,Y}(x,y) = p (X=x\ \mathrm {and} \ Y=y)
$$	


<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Multivariate_normal_sample.svg" />
</div>

* how does this simplify to intra-subject?
     * only rigid needed?


### Interpolation

* voxel w values - nearest neigh, linear, etc

### Additional from co-reg

Additional properties of the co-registration that we might want
* also would like to perform transformations backwards without computing the transforms again
* A mask in the target that one wants to place on the original individual brain image
    * 4x4 matrix inverse gives the inverse linear transformation
    
$$ v_{old} = T^{-1} v_{new} $$
    
    * non-linear mapping is not that easy to invert? -> some mechanism needed
        * one good way to do it is to estimage both directions simultaneously


## (time-)series preprocessing

Talk about how this is just a more simple application of the previous tools.

We have acquired a series of 3D volumes
In fMRI, the time-related contrast comes from blood oxygen-induced changes



In [16]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="fmri.mov">
</video>
""")


In dMRI, the contrast comes from diffusion sensitizing gradient-induced differences in the volumes. Signal in a gradient-specified direction is measured, the signal attenuates (disappears) faster in the directions where water molecules can diffuse more. And this gives an orientation selective contrast.
 


In [17]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="dwi.mov">
</video>
""")

Same brain, contrast etc: rigid co-registration except for susceptibility-induced artefacts or distance-to-the-receiver-related field inhomogeneities
    
fMRI & dMRI
        * show in video
* if the series contain subject motion, voxel (time-)series are not reliable
    * motion correction (with references to co-reg; rigid w CC-type metric)
        * rigid co-reg if no acquisition-induced distortions
            * video of pre and post
        * fMRI specific: slice-timing
            * 2D acquisitions - if slices are 
        * dMRI specific: eddy current correction
            * gradient changes induce eddy currents that can be corrected with specialiced algorigthms
        * any way to show the distortions?


In [18]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="mc_fmri.mov">
</video>
""")

## Spatial de-noising

dti enemmän

## Smoothing

Smoothing is performed for several reasons

Some co-registration algorightms work better with smoothed data

Smoothing removes small variations in multi-subject studies

Data becomes more gaussian after smoothing, which helps in statistical inference


In [19]:
from IPython.display import HTML
HTML("""
<video width="640" height="640" controls>
  <source src="smoothing.mov">
</video>
""")


How does the process look like (when using the tools presented earlier)

<img src="fMRI_pipeline.png" width="900" />

<img src="dMRI_pipeline.png" width="700" />


An example code in Snakemake formalism performing slice timing and motion correction to a dataset ../derivatives/{dataset}_EPI_ms.nii.gz, where {dataset} stands for any string. Slice timing is performed using FSL tool slicetimer and motion correction with ANTs tool antsMotionCorr.

```python
rule motion_correction:
    input:
        data="../derivatives/{dataset}_EPI_ms_st.nii.gz",
        reference="../derivatives/{dataset}_EPI_ms_st_reference.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st_mc.nii.gz"
    params:
        prefix="../derivatives/{dataset}_EPI_st"
    shell:
        "antsMotionCorr --dimensionality 3 --output [{params.prefix}_mc_,{output},{params.prefix}_ref.nii.gz] --transform Rigid[0.1] --metric GC[{input.reference},{input.data},1,32,Regular,0.2] --iterations 50x50 --useFixedReferenceImage 1 --useScalesEstimator 1 --smoothingSigmas 2x0 --shrinkFactors 2x1"

rule slice_timing:
    input:
        "../derivatives/{dataset}_EPI_ms.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st.nii.gz"
    shell:
        "slicetimer -i {input} -o {output} -r 1 --odd"
```

* show with images & videos
* voxel time-series before and after the process fig
* työkaluja löytyy
    * spesifisiä tai yleisiä
    * osassa graafiset käyttöliittymät
    * lista tähän



## Parametric maps from the data

### Diffusion tensor imaging (DTI)

The diffusion signal from the different gradient-weighted volumes is fitted into the 6-parameter exponential decay model

$$ S(g,b) = S_0 e^{-bg^TDg}, $$
where

$$
D =
\begin{pmatrix}
   D_{xx} & D_{xy} & D_{xz} \\
   D_{yx} & D_{yy} & D_{yz} \\
   D_{zx} & D_{zy} & D_{zz} 
\end{pmatrix}
$$

is the diffusion tensor, which can be visualized as an ellipsoid

<img src="ellipsoid.png" width="400" />,

where the eigenvalues of the rank-2 matrix can be used to compute diffusivity maps, such as Fractional Anisotropy FA

$$ \mathrm {FA} = \sqrt{.5 \frac{(\lambda_1-\lambda_2)^2 + (\lambda_1-\lambda_3)^2 + (\lambda_2-\lambda_3)^2}{\lambda_1^2 + \lambda_2^2 + \lambda_3^2}} $$

![FA](dwi_FA.png)

or Mean Diffusivity

![MD](dwi_MD.png)

    * eigen decomposition -> eigenvalues and vectors
    * eigenvalues -> parametric maps FA, MD show these






* T1 T1 - rekot

![T1](t1_img.png)
![flair](flair_img.png)
![flair_c](a_flair_img.png)
